In [1]:
import os
import sys
import json
import warnings

import numpy as np
from scipy import signal
import librosa

sys.path.insert(0, '/mnt/d/projects/bassline_extraction') 

import utilities as utils
from plotting import *
from transcription import *
from signal_processing import *

warnings.filterwarnings('ignore')
%matplotlib inline
np.set_printoptions(suppress=True)

# Initialization

## Directories

**TO DO: aligned_beat_positions => progression_beat_positions**

In [2]:
with open('../data/ouz_tracks.txt', 'r') as infile:
    track_titles = infile.read().split('\n')
       
_,_, bad_examples = next(os.walk('../data/bassline_extraction/beat_grid/bad_examples'))
bad_examples = [title.split('.txt')[0] for title in bad_examples]
       
with open('../data/metadata/scales_frequencies.json','r') as infile:
    scales = json.load(infile)

with open('../data/metadata/TechHouse_track_dicts.json','r') as infile:
    track_dicts = json.load(infile) 

# Listening and Waveform Inspection

Select a track to work with

utils.search_idx('Ashibah - Devotion (Extended Mix)', track_titles)

In [3]:
title = track_titles[17]
print('{}\n'.format(title))

if title in bad_examples:
    with open(os.path.join('../data/bassline_extraction/beat_grid/bad_examples',title+'.txt'), 'r') as infile:
        dpqb = float(infile.read())
    print("Beat grid problematic! dpqb: {:.2f}%\n{}".format(dpqb,title))
    
notes, scale_frequencies = utils.get_track_scale(title, track_dicts, scales)

fs = 44100
chorus, bassline, unprocessed_bassline = utils.load_audio(title) # read the chorus and the bassline
#utils.inspect_audio_outputs(title)

Billy Kenny & Huxley - Sweat



# Spectrogram Calculation

Calculate the spectrograms of the chorus and the bassline here.

You can also apply post filtering on the spectrogram.

In [4]:
n_fft = 4096*8
#win_length = 4096*4
win_length = int(fs*(60/125)/4) # quarter beat time resolution
#win_length = int(fs*(60/125)/8) # 1/8 beat time resolution

hop_length = int(win_length/2) 
assert win_length < n_fft, 'Window length must be greater than N_fft'

chorus_spectrogram = extract_dB_spectrogram(chorus, n_fft, win_length, hop_length)
bassline_spectrogram = extract_dB_spectrogram(bassline, n_fft, win_length, hop_length)

#  F0 Estimation 

## CREPE (Waveform Based)

Confidence level threshold filter on the initial F0 estimate

In [ ]:
F0_estimate, pitch_track = crepe_F0(bassline, 
                                     fs, 
                                     viterbi=True,
                                     threshold = 'mean_reduced') # get pitch track estimates

# Note Extraction

**Quantize the Pitch Track**:

in **time** to 1/4 th or 1/8 th beats

in **frequency** to Notes using Scale Information

In [ ]:
#N_samples = 3 # 1/16th beat
N_samples = 6 # number of 0.01 seconds intervals 1/8th beat
#N_samples = 12 # 1/4th beat

epsilon = 4 

# Count The Frequencies and Map to Notes for each interval
pitch_histograms = create_pitch_histograms(pitch_track, N_samples, epsilon, scale_frequencies) 
majority_pitches = get_majority_pitches(pitch_histograms) # Majority vote pitches for each interval

quantized_pitches = sample_and_hold(majority_pitches, N_samples) # single note to interval
        
pitch_track_quantized = (F0_estimate[0], quantized_pitches) # (time, freq)
bassline_notes = extract_notes(pitch_track_quantized, notes, scale_frequencies) # map the t, f quantized pitch track to notes 

# Plots

In [ ]:
plot_note_comparison(title, bassline_spectrogram, fs, hop_length, pitch_track, bassline_notes, save=False, plot_title='')

In [ ]:
utils.print_plot_play(chorus, fs, title)

In [ ]:
plot_spec_notes(title, bassline_spectrogram, fs, hop_length, bassline_notes_smoothed,
                  save=False, plot_title='_notes-quantized')

In [ ]:
utils.print_plot_play(bassline, fs, title)

## Spectral Maximum Power F0 Estimation

In [ ]:
# post filtering
#bassline_spectrogram_filtered = bassline_spectrogram.copy()
#bassline_spectrogram_filtered[bassline_spectrogram_filtered < -45.0] = -80.0

#_, bassline_frequencies = argmax_F0(bassline_spectrogram, fs, hop_length) # get argmax estimate 

F0_estimate = argmax_F0(bassline_spectrogram, fs, hop_length) # estimate the pitch track
plot_spec(title, bassline_spectrogram, fs, hop_length, F0=F0_estimate, save=False)

F0_estimate = argmax_F0(bassline_spectrogram, fs, hop_length) # F0 estimate
plot_wave_spec(title, bassline, bassline_spectrogram, fs, hop_length, F0=F0_estimate, save=False, plot_title='')

# Post Processing

## Pitch Shifting

In [ ]:
bassline_shifted = librosa.effects.pitch_shift(bassline, fs, 36)

bassline_shifted_spectrogram = extract_dB_spectrogram(bassline_shifted, n_fft, win_length, hop_length)

print_plot_play(bassline_shifted, fs)

In [ ]:
plot_spec(title, bassline_shifted_spectrogram, fs, hop_length, F0=True, save=False)

In [ ]:
freqs = [1, 256, 1024]

wc = [fc / (fs/2) for fc in freqs] # cutoff radians

lp = signal.firwin(5001, wc)

bassline_cut = signal.convolve(bassline_shifted, lp) # LP filter

bassline_cut = librosa.util.normalize(bassline_cut) # normalize

In [ ]:
bassline_shifted_spectrogram = extract_dB_spectrogram(bassline_cut, n_fft, win_length, hop_length)

In [ ]:
print_plot_play(chorus, fs, 'Original Chorus')

In [ ]:
plot_spec(title, bassline_shifted_spectrogram, fs, hop_length, F0=True, save=False)

## Energy Calculations

In [ ]:
title = track_titles[5]
print(title)

beat_positions = get_beat_positions(title)
bar_positions = get_bar_positions(beat_positions)

track = load_track(title, fs)[0]

energies = [np.sum(np.square(track[int(fs*bar_positions[idx]):int(fs*bar_positions[idx+1])])) for idx in range(len(bar_positions)-1)]

fig, ax = plt.subplots(figsize=(20,10) )

markerline, stemlines, baseline = ax.stem(range(len(energies)), energies, basefmt=" ")
markerline.set_markerfacecolor('b')
markerline.set_markersize(8)
stemlines.set_linewidth(0)

#plt.savefig('bar_energies')

In [ ]:
fc = 256

wc = fc / (fs/2) # cutoff radians

lp = signal.firwin(5001, wc)

track_cut = signal.convolve(track, lp) # LP filter

track_cut = librosa.util.normalize(track_cut) # normalize

In [ ]:
energies = [np.sum(np.square(track_cut[int(fs*bar_positions[idx]):int(fs*bar_positions[idx+1])])) for idx in range(len(bar_positions)-1)]

chunk_energies = []
for idx in range(len(energies)-3):
    
    if not idx%4:
        chunk_energies.append(np.mean(energies[idx:idx+4]))
    
extended = []
for en in chunk_energies:
    for i in range(4):
        
        extended.append(en)

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))

markerline, stemlines, baseline = ax.stem(range(len(extended)), extended, basefmt=" ")
markerline.set_markerfacecolor('b')
markerline.set_markersize(8)
stemlines.set_linewidth(0)
ax.vlines(8*np.arange(int(len(extended)/8 +1)), 0, 15000, alpha=0.8, color='c',linestyle='dashed', linewidths=3)
ax.set_title(title, fontsize=20)
ax.set_xlabel('Bars', fontsize=15)
ax.set_ylabel('Square Amplitude', fontsize=15)
plt.savefig('../figures/energy_levels/{}_energies_lp.png'.format(title))

plt.show()

fig, ax = plt.subplots(figsize=(20,10))

markerline, stemlines, baseline = ax.stem(range(len(energies)), energies, basefmt=" ")
markerline.set_markerfacecolor('b')
markerline.set_markersize(8)
stemlines.set_linewidth(0)
ax.vlines(16*np.arange(13), 0, 10000, alpha=0.8, color='c',linestyle='dashed', linewidths=3)

#plt.savefig('bar_energies_lp')

## other

frequency_bins, bin_width = create_frequency_bins(fs, n_fft)

# map frequencies to FFT bins
max_bins = []
for f in frequency:      
    bins = np.where(np.abs(f-frequency_bins)<bin_width)[0]
    
    if not bins.size> 0:
        print('no bin close')
    else:
        max_bins.append(bins[0])

In [ ]:
# silence frames
np.where(bassline_frequencies ==0)[0]

#non_silence = []
for idx, freq in enumerate(scale_frequencies):
    
    indices = np.where(np.abs(bassline_frequencies-freq)<3.0)[0]
    if indices.size>0:
        print(notes[idx])
    else:
        print(freq)
        
#non_silence = sorted(non_silence, reverse=False)

## Window Type for spectrogram

In [ ]:
window = signal.windows.blackmanharris(win_length)

amplitude_spectrogram = np.abs(librosa.stft(bassline, n_fft=n_fft, win_length=win_length, hop_length=hop_length, window=window))
dB_spectrogram = librosa.amplitude_to_db(amplitude_spectrogram, np.max(amplitude_spectrogram))  